In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [13]:
# Load API Credentials
with open('/Users/yawagipong/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)
    
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

In [14]:
location = 'Washington, DC'
term = 'Chicken'

In [15]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

In [16]:
JSON_FILE = FOLDER + f"{location.split(',')[0]}-{term}.json"
JSON_FILE

'Data/Washington-Chicken.json'

In [17]:
def create_json_file(JSON_FILE, delete_if_exists = False):
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == True:
        
        if delete_if_exists == True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file.")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE, delete_if_exists = False)
        
        else:
            
            print(f"[i] {JSON_FILE} already exists.")
            
    else:
        
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        folder = os.path.dirname(JSON_FILE)
        
        if len(folder) > 0:
            
            os.makedirs(folder, exist_ok = True)
            
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

In [19]:
create_json_file(JSON_FILE, delete_if_exists = True)

with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)

print(f' - {n_results} previous results found.')

results = yelp.search_query(location = location,
                               term = term,
                               offset = n_results)

total_results = results['total']

results_per_page = len(results['businesses'])

n_pages = math.ceil((total_results - n_results) / results_per_page)

n_pages

[!] Data/Washington-Chicken.json already exists. Deleting previous file.
[i] Data/Washington-Chicken.json not found. Saving empty list to new file.
 - 0 previous results found.


575

In [25]:
for i in tqdm_notebook( range(1,n_pages+1)):

    with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    results = yelp.search_query(location=location,
                                    term=term, 
                                    offset=n_results)

    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/575 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [26]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,y9CQsjE1CSlo1hOle63YVQ,little-chicken-washington-3,Little Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/2ZI9sF...,False,https://www.yelp.com/biz/little-chicken-washin...,99,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.0,"{'latitude': 38.90406531034134, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1100 15th St NW', 'address2': ''...",+12029890292,(202) 989-0292,2184.520495
1,gFxIqs_GjtqUfqg812ZA1Q,honeymoon-chicken-washington,Honeymoon Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/qhRM9Q...,False,https://www.yelp.com/biz/honeymoon-chicken-was...,216,"[{'alias': 'comfortfood', 'title': 'Comfort Fo...",4.0,"{'latitude': 38.94205356938236, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '4201 Georgia Ave NW', 'address2'...",+12029835010,(202) 983-5010,5540.239475
2,awfmaYM8dNNCYqlZBGCygA,roaming-rooster-washington-7,Roaming Rooster,https://s3-media1.fl.yelpcdn.com/bphoto/9wwCsE...,False,https://www.yelp.com/biz/roaming-rooster-washi...,48,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.5,"{'latitude': 38.949980000000004, 'longitude': ...",[],NaN,"{'address1': '4600 Wisconsin Ave NW', 'address...",+12024505933,(202) 450-5933,8568.640970
3,yezzRoh4t6Mxw1pPWP-SVQ,district-rico-washington-5,District Rico,https://s3-media2.fl.yelpcdn.com/bphoto/8e55QA...,False,https://www.yelp.com/biz/district-rico-washing...,318,"[{'alias': 'peruvian', 'title': 'Peruvian'}]",4.0,"{'latitude': 38.900365, 'longitude': -77.011819}","[delivery, pickup]",$$,"{'address1': '91 H St NW', 'address2': '', 'ad...",+12028425007,(202) 842-5007,844.760601
4,KudJEL3-xWdk_zVAwACmtQ,el-pollo-rico-arlington,El Pollo Rico,https://s3-media3.fl.yelpcdn.com/bphoto/Cs078k...,False,https://www.yelp.com/biz/el-pollo-rico-arlingt...,1622,"[{'alias': 'latin', 'title': 'Latin American'}...",4.5,"{'latitude': 38.88393729918841, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '932 N Kenmore St', 'address2': '...",+17035223220,(703) 522-3220,7556.805661


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,_HVdlE7-75fLOpxux1tpDg,14th-st-cafe-asian-bistro-washington-2,14th St Cafe Asian Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/ztRAAd...,False,https://www.yelp.com/biz/14th-st-cafe-asian-bi...,261,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 38.90943, 'longitude': -77.0322599}",[delivery],$$,"{'address1': '1416 14th St NW', 'address2': ''...",+12025880695,(202) 588-0695,2404.840437
996,irCj8hi2LXuhogOsqFAL3Q,charleys-cheesesteaks-washington-3,Charleys Cheesesteaks,https://s3-media2.fl.yelpcdn.com/bphoto/xs5F66...,False,https://www.yelp.com/biz/charleys-cheesesteaks...,25,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",4.0,"{'latitude': 38.89799359186031, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '50 Massachusetts Ave NE', 'addre...",+12022896566,(202) 289-6566,901.103221
997,QSzNSQqEbxtjBHEmNx7I2g,umaya-izakaya-washington-9,Umaya Izakaya,https://s3-media4.fl.yelpcdn.com/bphoto/5ZAO1B...,False,https://www.yelp.com/biz/umaya-izakaya-washing...,707,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",3.5,"{'latitude': 38.8987846374512, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '733 10th St NW', 'address2': Non...",+12022903443,(202) 290-3443,1193.521838
998,E4UAc8p9C6D0Rm7u2h3xYA,sandwiches-by-phillip-washington,Sandwiches by Phillip,https://s3-media2.fl.yelpcdn.com/bphoto/C-7sSk...,False,https://www.yelp.com/biz/sandwiches-by-phillip...,60,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.5,"{'latitude': 38.895625, 'longitude': -77.012355}",[],$$,"{'address1': '440 1st St NW', 'address2': None...",+12025253602,(202) 525-3602,332.083289
999,emy68nZERGuUmgKYTSsCPg,new-big-wong-chinese-restaurant-washington,New Big Wong Chinese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/UVm1oc...,False,https://www.yelp.com/biz/new-big-wong-chinese-...,563,"[{'alias': 'cantonese', 'title': 'Cantonese'},...",3.0,"{'latitude': 38.899623, 'longitude': -77.020772}","[pickup, delivery]",$$,"{'address1': '610 H St NW', 'address2': '', 'a...",+12026280491,(202) 628-0491,933.519919


In [27]:
final_df.duplicated(subset='id').sum()

77

In [28]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [29]:
final_df.to_csv('Data/final_results_DC_chicken.csv.gz', compression='gzip',index=False)